In [1]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/Appliance-Energy-Prediction")
import pandas as pd
from pca_fts.PcaSarimax import PcaSarimax
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
import datetime
import statistics
import math
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

## Aux functions

In [2]:
def sample_first_prows(data, perc=0.75):
    return data.head(int(len(data)*(perc)))

In [3]:
# convert series to supervised learning
def series_to_supervised_miso(data, n_in, n_out, endog_var, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [(df.columns[j]+'(t-%d)' % (i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df[endog_var].shift(-i))
		if i == 0:
			names += [(endog_var+'(t)')]
		else:
			names += [(endog_var+'(t+%d)' % (i))]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [4]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame) 
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [5]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

## Dataset Smart Home with Weather Information

In [6]:
df = pd.read_csv('/home/hugo/projetos-doutorado/Appliance-Energy-Prediction/data/HomeC.csv')
data = df.drop(labels=['icon','summary','cloudCover', 'House overall [kW]', 'time'], axis=1)
#data = data.loc[0: : 10]
data
data.dropna(inplace=True)
data = clean_dataset(data)
data.head()

/home/hugo/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,use [kW],gen [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],Garage door [kW],Kitchen 12 [kW],...,temperature,humidity,visibility,apparentTemperature,pressure,windSpeed,windBearing,precipIntensity,dewPoint,precipProbability
0,0.932833,0.003483,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
1,0.934333,0.003467,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,0.013117,0.000417,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
2,0.931817,0.003467,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,0.013083,0.000433,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
3,1.022050,0.003483,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,0.013000,0.000433,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
4,1.139400,0.003467,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,0.012783,0.000450,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0


## Dataset Smart Home with Weather Information, Lag = 1, t+1

In [7]:
data = series_to_supervised_miso(data, 1, 1,endog_var='use [kW]')
data.head()

,use [kW](t-1),gen [kW](t-1),Dishwasher [kW](t-1),Furnace 1 [kW](t-1),Furnace 2 [kW](t-1),Home office [kW](t-1),Fridge [kW](t-1),Wine cellar [kW](t-1),Garage door [kW](t-1),Kitchen 12 [kW](t-1),...,humidity(t-1),visibility(t-1),apparentTemperature(t-1),pressure(t-1),windSpeed(t-1),windBearing(t-1),precipIntensity(t-1),dewPoint(t-1),precipProbability(t-1),use [kW](t)
1,0.932833,0.003483,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,...,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0,0.934333
2,0.934333,0.003467,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,0.013117,0.000417,...,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0,0.931817
3,0.931817,0.003467,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,0.013083,0.000433,...,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0,1.022050
4,1.022050,0.003483,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,0.013000,0.000433,...,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0,1.139400
5,1.139400,0.003467,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,0.012783,0.000450,...,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0,1.391867


## Model: PcaSarimax - PCA-SARIMAX

### Define SARIMAX parameters 

In [7]:
pca_sarimax = PcaSarimax(n_components = 2,
                       endogen_variable = 'use [kW](t-1)',
                       order = [0, 0, 0],
                       seasonal_order = [0,0,0,0])

In [8]:
df_train = data.loc[:,'use [kW](t-1)':'precipProbability(t-1)']
train = sample_first_prows(data, perc=0.30)

In [9]:
reduced = pca_sarimax.apply_pca(train)
reduced.head(2)

,C0,C1,use [kW](t-1)
0,-0.137571,-0.220618,0.932833
1,-0.317482,0.483682,1.585083


In [10]:
exog = reduced.drop(labels=['use [kW](t-1)'], axis=1)
exog.head(2)

,C0,C1
0,-0.137571,-0.220618
1,-0.317482,0.483682


In [11]:
endog = reduced['use [kW](t-1)']
endog.head(3)

0    0.932833
1    1.585083
2    0.533817
Name: use [kW](t-1), dtype: float64

In [12]:
train_y = endog
train_X = exog

In [ ]:
# Define arimax paraters using the embedded data 
# from pmdarima.arima import auto_arima
# sarimax_model = auto_arima(train_y,
#                        exogenous= train_X,
#                        test='adf',
#                        start_p=0,
#                        start_q=0,
#                        max_p=5,
#                        max_q=5,
#                        start_Q=0,
#                        start_P=0,
#                        max_P=5,
#                        max_Q=5,
#                        m=7,
#                        seasonal=True,
#                        d=None,
#                        D=1,
#                        max_D=5,
#                        trace=True,
#                        error_action='ignore',
#                        suppress_warnings=True,
#                        stepwise=True)
# #                        n_fits = 50)

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,0)[7] intercept   : AIC=28245.212, Time=3.74 sec
 ARIMA(1,0,0)(1,1,0)[7] intercept   : AIC=21913.489, Time=23.21 sec
 ARIMA(0,0,1)(0,1,1)[7] intercept   : AIC=inf, Time=86.24 sec
 ARIMA(0,0,0)(0,1,0)[7]             : AIC=28243.214, Time=5.50 sec
 ARIMA(1,0,0)(0,1,0)[7] intercept   : AIC=25419.717, Time=7.34 sec
 ARIMA(1,0,0)(2,1,0)[7] intercept   : AIC=20519.374, Time=45.29 sec
 ARIMA(1,0,0)(3,1,0)[7] intercept   : AIC=19824.193, Time=58.00 sec
 ARIMA(1,0,0)(4,1,0)[7] intercept   : AIC=19427.974, Time=82.41 sec
 ARIMA(1,0,0)(5,1,0)[7] intercept   : AIC=19133.263, Time=137.18 sec
 ARIMA(1,0,0)(5,1,1)[7] intercept   : AIC=inf, Time=348.57 sec
 ARIMA(1,0,0)(4,1,1)[7] intercept   : AIC=inf, Time=210.98 sec


In [15]:
# Performing stepwise search to minimize aic
#  ARIMA(0,0,0)(0,1,0)[7] intercept   : AIC=28245.212, Time=3.74 sec
#  ARIMA(1,0,0)(1,1,0)[7] intercept   : AIC=21913.489, Time=23.21 sec
#  ARIMA(0,0,1)(0,1,1)[7] intercept   : AIC=inf, Time=86.24 sec
#  ARIMA(0,0,0)(0,1,0)[7]             : AIC=28243.214, Time=5.50 sec
#  ARIMA(1,0,0)(0,1,0)[7] intercept   : AIC=25419.717, Time=7.34 sec
#  ARIMA(1,0,0)(2,1,0)[7] intercept   : AIC=20519.374, Time=45.29 sec
#  ARIMA(1,0,0)(3,1,0)[7] intercept   : AIC=19824.193, Time=58.00 sec
#  ARIMA(1,0,0)(4,1,0)[7] intercept   : AIC=19427.974, Time=82.41 sec
#  ARIMA(1,0,0)(5,1,0)[7] intercept   : AIC=19133.263, Time=137.18 sec
#  ARIMA(1,0,0)(5,1,1)[7] intercept   : AIC=inf, Time=348.57 sec
#  ARIMA(1,0,0)(4,1,1)[7] intercept   : AIC=inf, Time=210.98 sec


### Test the model with the best parameters 

In [8]:
pca_arimax = PcaSarimax(n_components = 2,
                       endogen_variable = 'use [kW](t-1)',
                       order=[4,0,0],
                       seasonal_order=[5,1,0,7])

In [26]:
df_train = data.loc[:,'use [kW](t-1)':'precipProbability(t-1)']
df_test = data.loc[:,'gen [kW](t-1)':'use [kW](t)']
train = sample_first_prows(df_train,0.75)
test = df_test.iloc[max(train.index):]

In [9]:
# model, sarimax, pca_reduced_train = pca_arimax.run_train_model(train)

In [10]:
# sarimax.summary()

In [11]:
# pca_reduced_train.head(2)

In [12]:
#ALtera o nome da columa porque o modelo utiliza esse label como variável endogena 
# test.rename(columns = {'use [kW](t)': 'use [kW](t-1)'}, inplace = True)
# start = len(train)
# end = len(train) + len(test) -1
# forecast, pca_reduced_test = pca_arimax.run_test_model(test, sarimax,start,end)

In [13]:
# forecast

In [14]:
#pca_reduced_test['Appliances(t-1)'].head(2)
# pca_reduced_test.head(5)

In [15]:
# print(Measures.rmse(pca_reduced_test['use [kW](t-1)'],forecast))
# print(Measures.mape(pca_reduced_test['use [kW](t-1)'],forecast))

## Model: PcaSarimax - PCA-SARIMAX - Final model 

In [ ]:
#ARIMA(1,0,0)(1,1,0)[7] ==> 1.372592	0.833335	327.091827	34.337999      
#ARIMA(0,0,1)(0,1,1)[7] ==> 1.298641	0.729507	291.425668	28.129435
#ARIMA(1,0,0)(0,1,1)[7] ==> 1.284234	0.716202	292.696034	26.853493
#ARIMA(1,0,0)(1,0,0)[7] ==> 0.996471	0.579249	274.727737	27.777424
#ARIMA(1,0,0)(1,0,1)[7] ==> 0.994874	0.577666	272.759379	27.534379
#ARIMA(2,0,0)(1,0,0)[7] ==> 1.293463	0.728866	297.1769	28.026641
#ARIMA(1,0,1)(1,0,1)[7] ==> 1.290961e+65	1.468332e+64	1.069531e+66	29.484988
#ARIMA(1,0,0)(2,0,1)[7] ==> 0.994599	0.577157	272.337254	27.458885
#ARIMA(1,0,0)(2,0,1)[7] ==> 0.99471	0.577162	272.186821	27.424954
#ARIMA(1,0,0)(3,0,2)[7] ==> 0.99471	0.577162	272.186821	27.424954
#ARIMA(1,0,0)(4,0,2)[7] ==> 0.994843	0.57759	272.146911	27.471431

In [10]:
pca_sarimax = PcaSarimax(n_components = 3,
                       endogen_variable = 'use [kW](t-1)',
                       order=[1,0,0],
                       seasonal_order=[2,0,1,7])

## Sliding Window PyFTS: PCA-SARIMAX

In [11]:
result = {
     "window": [],
     "rmse": [],
     "mae": [],
     "mape": [],
     "smape": [],
     "nrmse":[]
}

tam = len(data)
n_windows = 30
windows_length = math.floor(tam / n_windows)
for ct, ttrain, ttest in Util.sliding_window(data, windows_length, 0.75, inc=1):
    if len(ttest) > 0:

        data_train = ttrain.loc[:,'use [kW](t-1)':'precipProbability(t-1)']
        data_test = ttest.loc[:,'gen [kW](t-1)':'use [kW](t)']
        
        #ALtera o nome da columa porque o modelo utiliza esse label como variável endogena 
        data_test.rename(columns = {'use [kW](t)': 'use [kW](t-1)'}, inplace = True)
        
        start = len(data_train)
        end = len(data_train)+ len(data_test) -1
        #appliance = ttest['Appliances_t+1'].reset_index()
        
        print('-' * 20)
        print(f'training window {(ct)}')
        model, sarimax, pca_reduced_train = pca_sarimax.run_train_model(data_train)
        forecast, pca_reduced_test = pca_sarimax.run_test_model(data_test,sarimax,start,end)
        
        print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics")
        rmse = Measures.rmse(pca_reduced_test['use [kW](t-1)'],forecast)
        mape = Measures.mape(pca_reduced_test['use [kW](t-1)'],forecast)
        smape = Measures.smape(pca_reduced_test['use [kW](t-1)'],forecast)
        
#         forecast = pd.DataFrame(forecast)
#         forecast.fillna(forecast.mean(),inplace=True)
#         forecast = np.array(forecast).reshape(-1)
        mae = mean_absolute_error(pca_reduced_test['use [kW](t-1)'], forecast)
    
        # Grafico de comparação Original x Previsto para cada janela
#         fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#         ax.plot(pca_reduced_test['use [kW](t-1)'], label='Original')
#         ax.plot(forecast, label='Forecast')
#         handles, labels = ax.get_legend_handles_labels()
#         lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#         plt.show()
        
        
        nrmse = cal_nrmse(rmse, pca_reduced_test['use [kW](t-1)'])
                
        result["rmse"].append(round(rmse,3))
        result["mae"].append(round(mae,3))
        result["mape"].append(round(mape,3))
        result["smape"].append(round(smape,3))
        result["nrmse"].append(round(nrmse,3))
        result["window"].append(ct)
        
measures = pd.DataFrame(result)

/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


--------------------
training window 0


/home/hugo/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


[ 14:00:41] getting statistics
--------------------
training window 16796


/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


...

[ 17:17:47] getting statistics
--------------------
training window 470288


/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/home/hugo/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


[ 17:22:47] getting statistics
--------------------
training window 487084


/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/home/hugo/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


[ 17:26:55] getting statistics


In [12]:
measures

,window,rmse,mae,mape,smape,nrmse
0,0,1.073,0.710,93.176,24.113,0.237
1,16796,0.415,0.215,89.842,12.081,0.053
2,33592,1.407,0.757,457.184,45.222,0.358
3,50388,0.703,0.349,31.754,16.481,0.116
4,67184,0.316,0.156,44.390,11.066,0.049
5,83980,0.288,0.202,84.772,19.921,0.079
6,100776,0.395,0.209,40.923,14.559,0.069
7,117572,0.449,0.196,28.295,11.636,0.073
8,134368,0.342,0.222,127.938,21.792,0.072
9,151164,0.503,0.188,62.510,13.490,0.043


In [12]:
#measures.to_csv (r'win_pca_sarimax_ksh_10min.csv', index = False, header=True)

In [13]:
measures.to_csv (r'win_pca_sarimax_ksh_1min.csv', index = False, header=True)

In [14]:
### Mean Statistics PCA-SARIMAX

In [15]:
final_result = {
     "rmse": [],
     "mae": [],
     "mape": [],
     "smape": [],
     "nrmse":[],
     "rmse_std": [],
     "mae_std": [],
     "mape_std": [],
     "smape_std": [],
     "nrmse_std": []
}

final_result["rmse"].append(round(statistics.mean(measures['rmse']),3))
final_result["mape"].append(round(statistics.mean(measures['mape']),3))
final_result["smape"].append(round(statistics.mean(measures['smape']),3))
final_result["mae"].append(round(statistics.mean(measures['mae']),3))
final_result["nrmse"].append(round(statistics.mean(measures['nrmse']),3))

final_result["rmse_std"].append(round(statistics.stdev(measures['rmse']),3))
final_result["mape_std"].append(round(statistics.stdev(measures['mape']),3))
final_result["smape_std"].append(round(statistics.stdev(measures['smape']),3))
final_result["mae_std"].append(round(statistics.stdev(measures['mae']),3))
final_result["nrmse_std"].append(round(statistics.stdev(measures['nrmse']),3))
        
final_measures_pca = pd.DataFrame(final_result)

print("Mean Statistics PCA-SARIMAX (test): ")
final_measures_pca

Mean Statistics PCA-SARIMAX (test): 


,rmse,mae,mape,smape,nrmse,rmse_std,mae_std,mape_std,smape_std,nrmse_std
0,0.587,0.309,115.044,20.121,0.095,0.347,0.184,108.335,8.31,0.067


In [16]:
#final_measures_pca.to_csv (r'pca_sarimax_ksh_10mim.csv', index = False, header=True)

In [16]:
final_measures_pca.to_csv (r'pca_sarimax_ksh_1mim.csv', index = False, header=True)